### Import libraries

In [ ]:
from ipywidgets import interactive
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import scipy.signal as sps
from IPython.display import HTML

In [ ]:
class signal:
    time = np.array([])
    amp = np.array([])

### Parameters

In [ ]:
signal_types = ['Short Rectangular pulse', 'Long Rectangular pulse', 
                'Trapesium','Triangular pulse', 'Ramp', 
                'Two Impulses', 'Three Impulses']

# Initialise variables
displacement = 0
signal1 = signal_types[0]

# Trigger graph updates
signal2 = signal_types[0]

step = 0.005

### Function Definitions

In [ ]:
def defSignal(sigName, signal_types):
    
    s = signal()
    s.time = np.arange(0)
    s.amp = np.arange(0)
    
    if sigName == signal_types[0]:
        s.time = np.arange(0, 6)
        s.amp = np.ones(s.time.size)
    elif sigName ==  signal_types[1]:
        s.time = np.arange(0, 16)
        s.amp = np.ones(s.time.size)
    elif sigName == signal_types[2]:
        s.time = np.arange(0, 24)
        s.amp = np.zeros(24)
        s.amp[0:4] = np.arange(0.2, 0.8, 0.2)
        s.amp[4:20] = 1
        s.amp[20:24] = np.arange(0.8, 0.2, -0.2)
    elif sigName == signal_types[3]:
        s.time = np.arange(0, 19)
        s.amp = np.zeros(19)
        s.amp[0:10] = np.arange(1, 11)
        s.amp[10:19] = np.arange(9, 0, -1)
    elif sigName == signal_types[4]:
        s.time = np.arange(0, 10)
        s.amp = np.arange(1, len(s.time)+1)
    elif sigName == signal_types[5]:
        s.time = np.arange(0, 15)
        s.amp = np.zeros(len(s.time))
        s.amp[0] = 1
        s.amp[-1] = 0.5
    elif sigName == signal_types[6]:
        s.time = np.arange(0, 15)
        s.amp = np.zeros(len(s.time))
        s.amp[0] = 1
        s.amp[int(np.round(len(s.time)/2))] = 0.5
        s.amp[-1] = 0.25 
    
    return s

In [ ]:
def Perform_animation(Signal1, Signal2):
    
    fig = plt.figure(figsize = (16, 8))
    
    ### If any of the signals has changed, update all of the signal graphs.
    
    # Get the signal data based on the signal name
    s1 = defSignal(Signal1, signal_types)
    s2 = defSignal(Signal2, signal_types)
    
    # Adjust the axes to account for the signal time axes
    disp_axes = np.arange(-len(s2.time), (len(s1.time)+len(s2.time)))
    
    # Plot signal 1
    ulaxes = plt.subplot(421)
    ulaxes.set_ylim(0, max(s1.amp)+0.1)
    ulaxes.set_xlabel('Sample, $n$')
    ulaxes.set_ylabel('Amplitude')
    ulaxes.set_title('Signal 1, x(n)')
    
    (marker_ul, stem_ul, base_ul) = plt.stem(s1.time, s1.amp, markerfmt = 'D')
    plt.setp(base_ul, color = 'black')
    plt.setp(stem_ul, color = 'blue')
    plt.setp(marker_ul, color = 'blue') 
    marker_ul.set_markerfacecolor('none')
    
    # Plot signal 2
    uraxes = plt.subplot(422)
    uraxes.set_ylim(0, max(s2.amp)+0.1)
    uraxes.set_xlabel('Sample, n')
    uraxes.set_ylabel('Amplitude')
    uraxes.set_title('Signal 2, y(n)')
    
    (marker_ur, stem_ur, base_ur) = plt.stem(s2.time, s2.amp, markerfmt = 'x')
    plt.setp(base_ur, color = 'black')
    plt.setp(stem_ur, color = 'red')
    plt.setp(marker_ur, color = 'red') 
    marker_ur.set_markerfacecolor('none')
 
    ### and replace the row graphs
    
    ## Plot second row graph 
    r2axes = plt.subplot(412)
    r2axes.margins(0.05)
    r2axes.set_ylim(0, max(max(s1.amp), max(s2.amp))+0.1)
    r2axes.set_xlabel('Sample, n ')
    r2axes.set_ylabel('Amplitude')
    r2axes.set_title('x(n) and y(n-l)')

    r2axes.set_xlim([min(disp_axes), max(disp_axes)])
    (marker_r2b, stem_r2b, base_r2b) = plt.stem(s1.time, s1.amp, markerfmt = 'D')
    plt.setp(base_r2b, color = 'black')
    plt.setp(stem_r2b, color = 'blue')
    plt.setp(marker_r2b, color = 'blue') 
    marker_r2b.set_markerfacecolor('none')
    
    # plot signal 2 on the display,
    (marker_r2r, stem_r2r, base_r2r) = plt.stem(displacement - s2.time, s2.amp, markerfmt = 'x')
    plt.setp(base_r2r, color = 'black')
    plt.setp(stem_r2r, color = 'red')
    plt.setp(marker_r2r, color = 'red') 
    marker_r2r.set_markerfacecolor('none')
    
    ## Plot third graph
    
    # Compute the convolution result
    conv_result_amp = np.correlate(s1.amp, s2.amp, 'full')
    
    # And its corresponding time series
    conv_result_time = np.arange(min(s1.time)+min(s2.time), 
                                 max(s1.time)+max(s2.time)+1)
    
    # Define the YLimits for the product graph
    product_limits = [min([max(s1.amp)*min(s2.amp), min(s1.amp)*max(s2.amp),0])-0.1, 
                      max([max(s1.amp)*max(s2.amp), min(s1.amp)*min(s2.amp),0])+0.1]
    
    product_s1 = np.zeros(len(disp_axes))
    product_s1[len(s2.time)+np.arange(len(s1.time))] = s1.amp
    product_s2 = np.zeros(len(disp_axes))
    product_s2[(np.arange(len(s2.time)) + displacement - 
               conv_result_time[0] + 1).astype(int)] = np.flip(s2.amp, axis = 0)
    product = np.multiply(product_s1, product_s2)
    
    r3axes = plt.subplot(413)
    r3axes.set_xlabel('Sample, n')
    r3axes.set_ylabel('Amplitude')
    r3axes.set_title('Product, x(n)y(n-l)')
    
    r3axes.set_xlim(min(disp_axes)-0.1, max(disp_axes)+0.1)
    r3axes.set_ylim(product_limits)
    
    (marker_r3, stem_r3, base_r3) = plt.stem(disp_axes, product, markerfmt ='s')
    plt.setp(base_r3, color = 'black')
    plt.setp(stem_r3, color = 'green')
    plt.setp(marker_r3, color = 'green') 
    marker_r3.set_markerfacecolor('none')
    
    ## Plot fourth graph
    # Set the axes extent, and plot the final result
    
    r4axes = plt.subplot(414)
    r4axes.set_ylim(0, max(conv_result_amp)+0.1)
    r4axes.set_xlabel('Sample, n')
    r4axes.set_ylabel('Amplitude')
    r4axes.set_title('Convolution')
    r4axes.set_xlim([min(conv_result_time)-0.1, 
                     max(conv_result_time)+0.1])
    
    (marker_r4, stem_r4, base_r4) = plt.stem(conv_result_time, conv_result_amp)
    plt.setp(base_r4, color = 'black')
    plt.setp(stem_r4, color = 'blue')
    plt.setp(marker_r4, color = 'blue') 
    marker_r4.set_markerfacecolor('none')
    
    (marker_r4g, stem_r4g, base_r4g)  = plt.stem([displacement], 
                                                 [conv_result_amp[displacement - conv_result_time[0]]])
    plt.setp(base_r4g, color = 'black')
    plt.setp(stem_r4g, color = 'green')
    plt.setp(marker_r4g, color = 'green') 
    marker_r4g.set_markerfacecolor('none')
    
    plt.tight_layout()
    
    def animate(i):
        value = (i) * step
        min_val = conv_result_time[0]
        max_val = conv_result_time[-1]
        displacement = max(min(min_val + np.round(value * (max_val - min_val) -0.5), max_val), min_val)
        
        ### Update row 2, 3, 4 graphs
        
        # The titles of rows 2 and 3 are functions of the displacement
        r2axes.set_title('$x(n)$ and $y(%d - n)$'%displacement)
        marker_r2r.set_xdata(displacement - s2.time)
        for i in range(len(stem_r2r)):
            stem_r2r[i].set_xdata(displacement - s2.time[i])

        # For row 3, the title is updated, and the product needs to be
        # computed each time the displacement changes
        r3axes.set_title('$x(n)y( %d -n)$'%displacement)
        
        product_s1 = np.zeros(len(disp_axes))
        product_s1[len(s2.time)+np.arange(len(s1.time))] = s1.amp
        product_s2 = np.zeros(len(disp_axes))
        product_s2[(np.arange(len(s2.time)) + displacement - 
                   conv_result_time[0] + 1).astype(int)] = np.flip(s2.amp, axis = 0)
        product = np.multiply(product_s1, product_s2)
        
        marker_r3.set_ydata(product)
        for i in range(len(stem_r3)):
            stem_r3[i].set_ydata([0, product[i]])

        # For row 4, only the single displacement result is plotted on
        # top of the existing convolution result
        stem_r4g[0].set_data([displacement, displacement],
                          [0, conv_result_amp[int(displacement - conv_result_time[0])]])
        marker_r4g.set_data(displacement, conv_result_amp[int(displacement - conv_result_time[0])])
        
        plt.tight_layout()
        

    ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                             frames = int(1/step)+1)
    plt.close()

    display(HTML(ani.to_jshtml()))



In [ ]:
### if cell show result in scrolling mode, click menu cell - Current Outputs - Toggle twice
matplotlib.rcParams.update({'font.size': 15})
interactive(Perform_animation, Signal1 = signal_types, Signal2 = signal_types)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.